In [19]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.dt_agent import DTAgent
from sklearn.model_selection import train_test_split

In [20]:
SEED = 42
np.random.seed(SEED)

#### The data

In [21]:
# df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_2.csv')
# df = df.fillna(-1)
# df.head()
test_df = pd.read_csv('../../data/more_features/train_sets/test_set_constant.csv')
test_df = test_df.fillna(-1)
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.391136,-1.000000,1.504298,5.058881,467.197112,96.252926,217.860499,3.550376,1,-1.000000,36.703810,122.310168,49.897027,20.353251,-1.000000,34.173407,46.631388,6
1,10.355048,272.506966,-1.000000,0.000000,128.706486,101.912313,-1.000000,3.048223,1,-1.000000,-1.000000,-1.000000,29.327349,-1.000000,-1.000000,31.065144,-1.000000,2
2,11.159385,-1.000000,-1.000000,0.000000,-1.000000,103.395616,77.212369,3.237870,0,-1.000000,8.690122,65.554731,19.167967,7.239049,-1.000000,33.478155,-1.000000,2
3,8.179735,140.876632,5.484515,-1.000000,303.740826,99.877458,155.573175,2.456931,0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
4,9.916825,-1.000000,3.909810,-1.000000,-1.000000,90.543986,189.347916,3.285748,0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,-1.000000,5


In [22]:
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)

#### The DT agent

In [23]:
%%time
dt_agent = DTAgent(X_test, y_test)
test_df = dt_agent.test()
test_df.head()

Testing done.....
Wall time: 4min 10s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Aplastic anemia]",6.0,6.0
1,4.0,1.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Unspe...",2.0,2.0
2,4.0,2.0,1.0,1.0,0.0,"[hemoglobin, mcv, segmented_neutrophils, Unspe...",2.0,2.0
3,4.0,3.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0
4,4.0,4.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0


In [7]:
# %%time
# dt_agent.test_sample(2)

In [8]:
# from datetime import datetime
# start = datetime.now()
# dt_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [9]:
# X_test_df.head()

In [8]:
len(test_df), len(X_test)

(14000, 14000)

In [9]:
# test_df.to_csv('../../test_dfs/many_features/0.1/correlated/dt_test_df_corr.csv', index=False)

In [10]:
# test_df = pd.read_csv('../../test_dfs/many_features/dt_test_df_0.1.csv')
# test_df.head()

In [11]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

100.0

In [12]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.006857142857143, 1.0)

In [13]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(1.0, 1.0, 1.0)

In [ ]:
# utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])